In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'coding-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4609245%2F7858108%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240610%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240610T131159Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D32e2c93bb23bd4b4d00ecc5af3ca0bc01ef5afd589465a445082dcf832d7330c79e6c15241552b4656c24ed2b57103203d889af72d0a3bc77b509dfa9201e2f9c20937975af17e2d50efbdfea9bd965cfbed4e8f57e4c8b67ebb5c881756ae7eb8f92b878e25d29be37213b6de807a3e6b654901a8814c978ba80e0eb256d10d5312e4f91795d3f395b77f070df32fbfd06bab7243417496fa5d2d2d235429d7edb461f0a667cdfba4ce40f626479f7837af07ffb73e41e2e24180eb7dd46847543172c595eb01be75a0b6bf9e83cb6f0356366e9b49bc0175c6465a1c4bb4884b052509eb7a6d90c9601d18b4d92bed459dcc2a22b401ca83a7ca4cd78257de'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# <b><span style='color:#F1A424'>|</span> Install Libraries</b><a class='anchor' id='install_libraries'></a> [↑](#top)

***

Install all the required libraries for this notebook.

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.


# <b><span style='color:#F1A424'>|</span> Import Libraries</b><a class='anchor' id='import_libraries'></a> [↑](#top)

***

Import all the required libraries for this notebook.

In [3]:
# Import necessary libraries
import pandas as pd
from tqdm import tqdm


import os

import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from accelerate import Accelerator

# import numpy as np
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTTrainer, DataCollatorForCompletionOnlyLM
import sklearn
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

2024-06-20 18:37:30.052458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 18:37:30.052598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 18:37:30.204247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# <b><span style='color:#F1A424'>|</span> Load custom dataset</b><a class='anchor' id='load_data'></a> [↑](#top)

***

Custom dataset is used in this notebook. You can use any data but dataset should contain two columns with name 'prompt' and 'response'. The prompt column should contain the input text.

In [4]:
model_name_or_path = "NousResearch/llama-2-7b-chat-hf"

In [5]:

df = pd.read_csv("/kaggle/input/sample-problems/data_leetcode_problems.csv")
df.drop('Unnamed: 0',axis = 1,inplace=True)
train_df, eval_df = train_test_split(df, test_size = 0.15, random_state = 42)

train_ds = Dataset.from_pandas(train_df, split="train")
eval_ds = Dataset.from_pandas(eval_df, split="test")



tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"
train_ds = train_ds.flatten()
eval_ds = eval_ds.flatten()


def tokenize(df):
    result = tokenizer(
        df['text'],
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result


# def tokenize_function(example_batch):
#     input_encodings = tokenizer(example_batch["input"], max_length=1024, truncation=True, padding=True)
#     with tokenizer.as_target_tokenizer():
#         target_encodings = tokenizer(example_batch["output"], max_length=128, truncation=True, padding=True)
#     return {
#         "input_ids": input_encodings["input_ids"],
#         "attention_mask": input_encodings["attention_mask"],
#         "labels": target_encodings["input_ids"]
#     }
# tokenized_train_data = train_ds.map(tokenize_function,batched = True)
# tokenized_eval_data = eval_ds.map(tokenize_function, batched = True)


# def preprocess_function(examples):
#     if "instruction" in examples:  # Check if the column exists
#         return tokenizer([" ".join(x) for x in examples["instruction"]])
#     else:
#     # Handle the case where the column is missing (e.g., raise an error or use a default value)
#         raise ValueError("Instruction column not found in the dataset")
#         return tokenizer([" ".join(x) for x in train_ds["output"]])
# tokenizer_train_ds = train_ds.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=train_ds.column_names,
# )
# tokenizer_eval_ds = eval_ds.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=train_ds.column_names,
# )

tokenized_train_dataset = train_ds.map(tokenize,batched=True,remove_columns=['instruction','input','output'])
tokenized_eval_dataset = eval_ds.map(tokenize,batched=True,remove_columns=['instruction','input','output'])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Map:   0%|          | 0/17825 [00:00<?, ? examples/s]

Map:   0%|          | 0/3146 [00:00<?, ? examples/s]

# <b><span style='color:#F1A424'>|</span> Configuration of Quantization and LORA parameters</b><a class='anchor' id='configure_parameters'></a> [↑](#top)

***

As model size is big it is loaded in 4 bit.

In [6]:
model_name = "NousResearch/llama-2-7b-chat-hf"
# dataset_name = "/content/train.jsonl"
# new_model = "llama-2-7b-custom"
# lora_r = 64
# lora_alpha = 16
# batch_size =128
# lora_dropout = 0.1
# use_4bit = True
# bnb_4bit_compute_dtype = "float16"
# bnb_4bit_quant_type = "nf4"
# use_nested_quant = False
# output_dir = "./results"
# num_train_epochs = 1  # Reduced epochs
# fp16 = True #mde this true so that precision is changed
# bf16 = False
# per_device_train_batch_size = 2  # reduced batch size for fitting into memory batch size
# per_device_eval_batch_size = 2
# gradient_accumulation_steps = batch_size // per_device_train_batch_size # Adjust gradient accumulation
# gradient_checkpointing = True
# max_grad_norm = 0.2
# learning_rate = 3e-4
# weight_decay = 0.001
# optim = "adamw_torch"
# lr_scheduler_type = "constant"
# max_steps = 100
# warmup_ratio = 0.03
# group_by_length = False
# save_steps = 15
# logging_steps = 5
# max_seq_length = None
# packing = False
device_map = "auto" # Utilize both GPUs

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 1
num_train_epochs = 4
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 70
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [8]:
# Configure quantization parameters
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load pre-trained model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
)
model.config.use_cache = False
# model.config.pretraining_tp = 1



# Configure LoRA-specific parameters
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# eval_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# Create a solution in python for the input asked.

# ### Input:
# The algorithm leverages a hash map (unordered_map in C++, HashMap in Java, dictionary in Python, and Map in JavaScript). It iterates through the given 'nums' array and calculates the complementary value (target - current value). If the complementary value is already in the hash map, it means that we found a solution, and we return those indices. If the complement is not in the hash map, we store the current element in the hash map with its index. If the algorithm doesn't find the solution, it returns an empty array or throws an exception (in Java).

# This approach has a time complexity of O(n) and a space complexity of O(n) as well.

# ### Response:
# """
# # {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500)[0], skip_special_tokens=True))

# <b><span style='color:#F1A424'>|</span> Training</b><a class='anchor' id='training'></a> [↑](#top)

***


In [9]:
if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

In [10]:
resume_from_checkpoint = "adapter_model.bin" # set this to the adapter_model.bin file you want to resume from

if resume_from_checkpoint:
    if os.path.exists(resume_from_checkpoint):
        print(f"Restarting from {resume_from_checkpoint}")
        adapters_weights = torch.load(resume_from_checkpoint)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {resume_from_checkpoint} not found")

Checkpoint adapter_model.bin not found


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model  # Ensure get_peft_model is imported
from transformers import BitsAndBytesConfig
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, DataCollatorForSeq2Seq
# accelerator = Accelerator()

# model = get_peft_model(model, peft_config)

# model.train()
# Data collator for language modeling
# model.gradient_checkpointing_enable() 


# for param in model.parameters():
#     if param.dtype in [torch.float16, torch.float32, torch.float64, torch.complex64, torch.complex128]:
#         param.requires_grad = True

# wandb_project = "sql-try2-coder"
# if len(wandb_project) > 0:
#     os.environ["WANDB_PROJECT"] = wandb_project

# Training arguments
training_args = TrainingArguments(
   output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=3,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type, # group sequences of roughly the same length together to speed up training # if use_wandb else "none",

)

# Trainer instantiation
# trainer = Trainer(
# #     model=model,
# #     args=training_args,
# #    train_dataset = tokenized_train_dataset,
# #     eval_dataset=tokenized_eval_dataset,
# #     tokenizer=tokenizer,
# #     data_collator=DataCollatorForSeq2Seq(
# #         tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
#     ),

max_seq_length = 2048
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)
# model = torch.compile(model)
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/17825 [00:00<?, ? examples/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.345800
2,1.852300
3,1.233600
4,1.444300
5,1.052300
6,1.648400
7,1.328100
8,1.080000
9,1.471300
10,1.062800


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=70, training_loss=0.9420680446284158, metrics={'train_runtime': 204.6483, 'train_samples_per_second': 0.342, 'train_steps_per_second': 0.342, 'total_flos': 396683740569600.0, 'train_loss': 0.9420680446284158, 'epoch': 0.0})

# <b><span style='color:#F1A424'>|</span> Testing</b><a class='anchor' id='testing'></a> [↑](#top)

***

Testing on test data

In [12]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)
     

In [18]:
print(eval_ds['text'][500])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a function in Python that takes two integers as arguments and returns the number of common divisors between the two.

### Input:
350, 400

### Response: 
def countCommonDivisors(a,b): 
   
    divisors = [] 
   
    for i in range(1, min(a,b)+1): 
        if (a % i == 0 and b % i == 0): 
            divisors.append(i) 
              
    return len(divisors) 

num1 = 350
num2 = 400

print (countCommonDivisors(num1, num2)) 
# Output: 5


In [14]:
eval_ds['output'][5]

'def bubbleSort(arr): \n    n = len(arr) \n  \n    for i in range(n):      \n        for j in range(0, n-i-1): \n            if arr[j] > arr[j+1] : \n                arr[j], arr[j+1] = arr[j+1], arr[j] \n  \n    return arr'

In [19]:
text = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a function in Python that takes two integers as arguments and returns the number of common divisors between the two.

### Input:
350, 400

### Response: 


"""
device = "cuda:0"

In [20]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a function in Python that takes two integers as arguments and returns the number of common divisors between the two.

### Input:
350, 400

### Response: 


{
    def common_divisors(a, b):
        return [x for x in range(1, min(a, b) + 1) if a ** x == b ** x]
}

This function takes two integers `a` and `b` as input and returns a list of common divisors between them. The function uses a brute force approach by iterating over all possible divisors between `a` and `b` and checking if their product is equal to the product of `a` and `b`.

For example, if `a = 350` and `b = 400`, the function will return the list `[100]`.

Note: This function assumes that the input integers are non-negative and less than or equal to `1000`. If the input integers can be arbitrary large, you may need to modify the function accord

# <b><span style='color:#F1A424'>|</span> Saving Model for inference</b><a class='anchor' id='save_model'></a> [↑](#top)

***


In [22]:
from huggingface_hub import login
login()
     

In [24]:
model.push_to_hub("icedveins23/python_problem_solving",create_pr=1)

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/icedveins23/python_problem_solving/commit/a86355dc5c7fd2fe7f97623a8fa6713adb2a7466', commit_message='Upload model', commit_description='', oid='a86355dc5c7fd2fe7f97623a8fa6713adb2a7466', pr_url='https://huggingface.co/icedveins23/python_problem_solving/discussions/1', pr_revision='refs/pr/1', pr_num=1)